In [1]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
import keras
import csv
import pandas as pd

In [82]:
def normalize(X_train):
   newx_train=tf.cast(X_train,tf.float32)/255
   
   return newx_train


In [9]:
def initialize_parameters():
    hidden_units=[(20,784),(15,20),(12,15),(10,12)]
    parameters=dict()
    
    layers=len(hidden_units)
    
    for i in range(1,layers+1):
        parameters["W"+str(i)]=tf.Variable(np.random.randn(hidden_units[i-1][0],hidden_units[i-1][1])*np.sqrt(2/np.power(i,(layers-1))),name="w"+str(i))
        parameters["B"+str(i)]=tf.Variable(np.random.randn(hidden_units[i-1][0],1),name="b"+str(i))
    return parameters
        

In [89]:
def convert_to_one_hot(label,depth=10):
    encoded=tf.one_hot(label,depth,axis=0)
    transformed_Y=tf.reshape(encoded,[depth,])
    
    return transformed_Y

In [11]:
def forward_propagation(X,parameters):
    
    layers=len(parameters)//2
    activations,linearOutput=dict()
    
    activations["A0"]=X
    for i in range(1,layers+1):
        linearOutput["Z"+str(i)]=tf.add(tf.linalg(parameters["W"+str(i)],activations["A"+str(i-1)]),parameters["B"+str(i)])
        if i!=layers-1:
            activations["A"+str(i)]=tf.keras.activations.relu(linearOutput["Z"+str(i)])
        
    return linearOutput["Z"+str(layers)]   

In [12]:
def losses(linear,y_labels):
    cost=tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_labels,linear))
    
    return cost


In [52]:
def model(X_train,Y_train,learning_rate=0.001,num_epochs=5000,minibatch_size=32,print_cost=True):
    costs=list()
    training_accuracies=list()
    
    parameters=initialize_parameters()
    optimizing_algo=tf.keras.optimizers.Adam(learning_rate)
    accuracy=tf.keras.metrics.CategoricalAccuracy()
    trainingSet=tf.data.Dataset.zip((X_train,Y_train))
    minibatches=trainingSet.batch(minibatch_size).prefetch(8)
    
    m=trainingSet.cardinality().numpy()
    
    for e in range(1,num_epochs+1):
        epochCost=0
        accuracy.reset_states()
        
        for (mini_X,mini_Y) in minibatches:
            
            with tf.GradientTape as t:
                linearOutput=forward_propagation(mini_X,parameters)
                
                mini_loss=losses(linearOutput,Y_train)
            accuracy.update_state(mini_Y,tf.transpose(linearOutput))
            
            training_parameters=list(parameters.keys())
            gradients=t.gradient(mini_loss,training_parameters)
            
            optimizing_algo.apply_gradients(zip(gradients,training_parameters))
            
            epochCost+=mini_loss
        epochCost/=m
        
        if print_cost==True and e%10==0:   
            print("Cost after epoch",e,"-",epochCost)
            print("Training accuracy - ",accuracy.result())
            
            costs.append(epochCost)
            training_accuracies.append(accuracy.result())
    return parameters,costs,training_accuracies,costs

In [70]:

#main
train_set=pd.read_csv("train.csv")
test_set=pd.read_csv("test.csv")

In [76]:
x_train=tf.data.Dataset.from_tensor_slices(train_set.drop('label',axis=1))
y_train=tf.data.Dataset.from_tensor_slices(train_set['label'])

In [77]:
x_train.element_spec

TensorSpec(shape=(784,), dtype=tf.int64, name=None)

In [78]:
print(next(iter(x_train)))

tf.Tensor(
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 188 255  94   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 191 250 253
  93   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 123 248 253 167  10   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  80 247 253
 208  13   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0  29 207 253 235

In [79]:
y_train.element_spec

TensorSpec(shape=(), dtype=tf.int64, name=None)

In [93]:
x_new_train=x_train.map(normalize)

In [94]:
y_new_train=y_train.map(convert_to_one_hot)

In [95]:
k=tf.data.Dataset.zip((x_new_train,y_new_train))

In [96]:
p=initialize_parameters()